In [61]:
import hashlib
import binascii
from bip_utils import (
    Bip39SeedGenerator, Bip32Slip10Secp256k1, Bip32Slip10Ed25519,
    Bip39MnemonicGenerator, Bip39Languages, Bip39WordsNum, Bip39MnemonicValidator, Bip39Mnemonic,
    Bip44Changes, Bip84Coins, Bip84
)
from pytezos.crypto.encoding import base58_encode
from pytezos.crypto import key
import pytezos
from eth_account import Account
from bitcoin.wallet import CBitcoinSecret, CBech32BitcoinAddress, P2WPKHBitcoinAddress, CBitcoinAddress
from bitcoin.core.key import CECKey, CPubKey
from bitcoin.core.script import CScript,OP_0
from bitcoin.core import Hash160
import requests

In [111]:
pytezos.pytezos.using

<function PyTezosClient.using at 0x7f9e2c0cbc70>
Change current RPC endpoint and account (private key).

:param shell: one of 'mainnet', '***net', or RPC node uri, or instance of :class:`pytezos.rpc.shell.ShellQuery`
:param key: base58 encoded key, path to the faucet file, faucet file itself, alias from tezos-client, or `Key`
:param mode: whether to use `readable` or `optimized` encoding for parameters/storage/other
:returns: A copy of current object with changes applied

# HD Wallets(public key, private key, address)

## Mnemonics

### Generate random mnemonic of twelve words

In [23]:
random_mnemonics = Bip39MnemonicGenerator(Bip39Languages.ENGLISH).FromWordsNumber(Bip39WordsNum.WORDS_NUM_12).ToStr()
random_mnemonics

'lyrics size wet garbage arrange cable round visual mixed route shuffle dutch'

### Validate Mnemonics

In [24]:
mnemonic = "invalid abandon abandon abandon abandon abandon abandon abandon abandon abandon abandon about"

# Get if a mnemonic is valid with automatic language detection, return bool
valid = Bip39MnemonicValidator(Bip39Languages.ENGLISH).IsValid(mnemonic)
# Same but specifying the language
not_valid = Bip39MnemonicValidator(Bip39Languages.ENGLISH).IsValid(random_mnemonics)

valid, not_valid

(False, True)

In [212]:
# mnemonic = "imitate embody law mammal exotic transfer roof hope price swift ordinary uncle"
mnemonic = 'chat style uncover beauty federal dish priority best then empty lesson flag'
seed_bytes = Bip39SeedGenerator(mnemonic).Generate()

In [213]:
TEZOS_PATH = "m/44'/1729'/0'/0'"
ETHEREUM_PATH = "m/44'/60'/0'/0/0"
BITCOIN_PATH_LEGACY = "m/44'/0'/0'/0/0"
BITCOIN_PATH_SEGWIT = "m/84'/0'/0'/0/0"

### Tezos Key Store

In [214]:
bip32_ctx = Bip32Slip10Ed25519.FromSeedAndPath(seed_bytes, TEZOS_PATH)

In [215]:
private_key_hex = bip32_ctx.PrivateKey().Raw().ToHex()

In [216]:
tezos_private_key = base58_encode(bytes.fromhex(private_key_hex), prefix=b'edsk')
tezos_private_key

b'edsk2xWEMfyhLzafUqvgRVx3PsjNkbZJpCuiVCeMyK8DK78S4SNJxd'

In [217]:
account = key.Key.from_encoded_key(tezos_private_key)
tezos_address = account.public_key_hash()
tezos_address

'tz1SzAmjGiyrW2joqwbHBo1jeNGHrjXoE5tq'

### Ethereum Key Store

In [218]:
bip32_ctx = Bip32Slip10Secp256k1.FromSeedAndPath(seed_bytes, ETHEREUM_PATH)

In [219]:
private_key_hex = bip32_ctx.PrivateKey().Raw().ToHex()

In [220]:
account = Account.from_key(private_key_hex)
ethereum_private_key = account.privateKey
ethereum_private_key

HexBytes('0xab4a0d5fcafaa7fb9fbe59690e0ae2ee6c94989a5c07da79c40bd152583d6900')

In [221]:
ethereum_address = account.address
ethereum_address

'0x1DEF760Eb8f81167dF93E27D8963a27895E3458f'

### Bitcoin Key Store

In [222]:
bip32_ctx = Bip32Slip10Secp256k1.FromSeedAndPath(seed_bytes, BITCOIN_PATH_SEGWIT)

In [223]:
private_key_bytes = bip32_ctx.PrivateKey().Raw().ToBytes()
private_key_bytes

b'9\xd3{k\x05O!\xe4\xf5F\x07\xc6\x0bD\x8b\xd7\xb5\xc2d \\\x92\xb4I\x81\xc4\xa43An.\x9b'

In [224]:
account = CBitcoinSecret.from_secret_bytes(private_key_bytes)

In [225]:
script_pubkey = CScript([OP_0, Hash160(account.pub)])
bitcoin_address = CBech32BitcoinAddress.from_scriptPubKey(script_pubkey)
bitcoin_address

P2WPKHBitcoinAddress('bc1q3qwydrdrq4tsfcf2yp5u55ja423p0wx6ss5mhu')

In [226]:
account.hex()

'39d37b6b054f21e4f54607c60b448bd7b5c264205c92b44981c4a433416e2e9b01'

# Nodes, Indexers, Explorers

## Block Explorers

Bitcoin, Ethereum, etc ...- https://blockchair.com/

Bitcoin - https://blockstream.info/

Ethereum - https://etherscan.io/

Tezos - https://tzkt.io/

## Indexers

Bitcoin - https://github.com/Blockstream/electrs

Ethereum - https://etherscan.io/

Tezos - https://api.tzkt.io/

In [167]:
# Tezos https://api.ghostnet.tzkt.io

r = requests.get(f'https://api.ghostnet.tzkt.io/v1/accounts/{tezos_address}/balance')
balance = int(r.text)/1000000

r = requests.get(f'https://api.ghostnet.tzkt.io/v1/accounts/{tezos_address}/operations')
history = r.json()

In [168]:
balance, history

(100.0,
 [{'type': 'transaction',
   'id': 67199478792192,
   'level': 1325697,
   'timestamp': '2022-10-13T03:17:45Z',
   'block': 'BM8oJMj35dDDuTKAGcryBQryG6pX1oHZQWhN1vpMYrxRzouKQkr',
   'hash': 'onxxzZXWVkDEi86xVqsN1wMwU8qTEieXmWWnKctHSaXv9jexpA1',
   'counter': 21482,
   'sender': {'address': 'tz1VSUr8wwNhLAzempoch5d6hLRiTh8Cjcjb'},
   'gasLimit': 1101,
   'gasUsed': 1001,
   'storageLimit': 257,
   'storageUsed': 0,
   'bakerFee': 402,
   'storageFee': 0,
   'allocationFee': 64250,
   'target': {'address': 'tz1Xi3qNt4bvNWDN1D1EmeNCddrDPZhC1zsM'},
   'amount': 100000000,
   'status': 'applied',
   'hasInternals': False}])

In [200]:
# Ethereum  https://api-sepolia.etherscan.io

r = requests.get(f'https://api-sepolia.etherscan.io/api?module=account&action=balance&address=0x382b4ca2c4a7cd28c1c400c69d81ec2b2637f7dd&tag=latest')
balance = int(r.json()['result'])/1000000000000000000

r = requests.get(f'https://api-sepolia.etherscan.io/api?module=account&action=txlist&address=0x382b4ca2c4a7cd28c1c400c69d81ec2b2637f7dd&startblock=0&endblock=99999999&page=1&offset=10&sort=asc')
history = r.json()
# https://sepolia.etherscan.io/

In [201]:
balance, history

(21.50238249997988,
 {'status': '0',
  'message': 'NOTOK',
  'result': 'Max rate limit reached, please use API Key for higher rate limit'})

In [199]:
# Bitcoin https://blockstream.info/testnet/api


21.50238249997988

## Nodes

### Ethereum

- https://infura.io
- https://www.alchemy.com/

### Tezos

- https://mainnet.api.tez.ie
- https://mainnet.smartpy.io	
- https://rpc.tzbeta.net/	

# Sending Crypto

Sending crypto requires 3 steps

1. Build transaction

2. Sign transaction

3. Broadcast transaction

## Tezos

In [129]:
pytezos.pytezos.using(shell="https://ghostnet.smartpy.io", key=account)


Properties
.key		tz1Xi3qNt4bvNWDN1D1EmeNCddrDPZhC1zsM
.shell		['https://ghostnet.smartpy.io']
.block_id	head

Helpers
.account()
.activate_account()
.activate_protocol()
.bake_block()
.balance()
.ballot()
.bulk()
.check_message()
.contract()
.delegation()
.double_baking_evidence()
.double_endorsement_evidence()
.endorsement()
.endorsement_with_slot()
.failing_noop()
.now()
.operation()
.operation_group()
.origination()
.proposals()
.register_global_constant()
.reveal()
.seed_nonce_revelation()
.sign_message()
.sleep()
.transaction()
.using()
.wait()

In [130]:
pytezos.pytezos.reveal().autofill().sign().inject()

RpcError: ({'contract': 'tz1grSQDByRpnVs7sPtaprNZRp531ZKz6Jmm',
  'id': 'proto.014-PtKathma.contract.previously_revealed_key',
  'kind': 'branch'},)

In [131]:
tx = pytezos.pytezos.transaction(destination='tz1R5X6pDpoxbMkY4AQJmUhRfEHjYN7dq66E', amount=10*1000000).autofill().sign().inject()

In [132]:
tx

{'chain_id': 'NetXnHfVqm9iesp',
 'hash': 'onuBmZ13KRw8CNYa6tu79vBMAQKQMeDbNcbRpYRPh7x4mqTPbax',
 'protocol': 'PtKathmankSpLLDALzWw7CGD2j2MtyveTwboEYokqUCP4a1LxMg',
 'branch': 'BMTQ21L3Ke46YtYYTH2gg2zq4iyRUBV8nhv91A1E5YY462gYTuf',
 'contents': [{'kind': 'transaction',
   'source': 'tz1grSQDByRpnVs7sPtaprNZRp531ZKz6Jmm',
   'fee': '373',
   'counter': '393388',
   'gas_limit': '1101',
   'storage_limit': '100',
   'amount': '10000000',
   'destination': 'tz1R5X6pDpoxbMkY4AQJmUhRfEHjYN7dq66E'}],
 'signature': 'sigsvGjkMX1P8zZh7r6gtTMxL7J9y4B2VWJHdEwyzMPCs7qDRGHT1QmbJWnSqw8rgd43jW9NSZ58wkosuutLFcfMMPitsfcF'}

In [133]:
tx['hash']

'onuBmZ13KRw8CNYa6tu79vBMAQKQMeDbNcbRpYRPh7x4mqTPbax'